## Contexto y objetivo del proyecto

Film Junky Union, una comunidad enfocada en el análisis de películas clásicas, busca desarrollar un sistema automático para **clasificar reseñas de películas según su polaridad**.

El objetivo de este proyecto es **entrenar un modelo de procesamiento de lenguaje natural (NLP)** capaz de identificar reseñas negativas a partir de texto, utilizando un conjunto de datos de reseñas de IMDB etiquetadas como positivas o negativas.

El desempeño del modelo se evalúa mediante la métrica **F1-score**, la cual debe alcanzar un valor mínimo de **0.85**, de acuerdo con los requisitos del proyecto.

### Nota sobre el uso de BERT

Adicionalmente, se incluye un modelo basado en **embeddings de BERT** como aproximación avanzada de NLP.

Debido a las limitaciones de procesamiento en CPU, el entrenamiento y evaluación de este modelo se realizaron utilizando únicamente **200 ejemplos de entrenamiento y 200 de prueba**, siguiendo las recomendaciones del curso.

## Inicialización

In [ ]:
import math

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

from tqdm.auto import tqdm

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
# la siguiente línea proporciona gráficos de mejor calidad en pantallas HiDPI
# %config InlineBackend.figure_format = 'retina'

plt.style.use('seaborn')

In [ ]:
# esto es para usar progress_apply, puedes leer más en https://pypi.org/project/tqdm/#pandas-integration
tqdm.pandas()

## Cargar datos

In [ ]:
df_reviews = pd.read_csv('/datasets/imdb_reviews.tsv', sep='\t', dtype={'votes': 'Int64'})

In [ ]:
df_reviews.info()

In [ ]:
print(df_reviews.head())

In [ ]:
print(df_reviews['ds_part'].value_counts())

In [ ]:
print(df_reviews['pos'].value_counts())

In [ ]:
# revisar valores ausentes
df_reviews.isna().sum()

In [ ]:
# revisar filas duplicadas
df_reviews.duplicated().sum()

## EDA

Veamos el número de películas y reseñas a lo largo de los años.

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(16, 8))

ax = axs[0]

dft1 = df_reviews[['tconst', 'start_year']].drop_duplicates() \
    ['start_year'].value_counts().sort_index()
dft1 = dft1.reindex(index=np.arange(dft1.index.min(), max(dft1.index.max(), 2021))).fillna(0)
dft1.plot(kind='bar', ax=ax)
ax.set_title('Número de películas a lo largo de los años')

ax = axs[1]

dft2 = df_reviews.groupby(['start_year', 'pos'])['pos'].count().unstack()
dft2 = dft2.reindex(index=np.arange(dft2.index.min(), max(dft2.index.max(), 2021))).fillna(0)

dft2.plot(kind='bar', stacked=True, label='#reviews (neg, pos)', ax=ax)

dft2 = df_reviews['start_year'].value_counts().sort_index()
dft2 = dft2.reindex(index=np.arange(dft2.index.min(), max(dft2.index.max(), 2021))).fillna(0)
dft3 = (dft2/dft1).fillna(0)
axt = ax.twinx()
dft3.reset_index(drop=True).rolling(5).mean().plot(color='orange', label='reviews per movie (avg over 5 years)', ax=axt)

lines, labels = axt.get_legend_handles_labels()
ax.legend(lines, labels, loc='upper left')

ax.set_title('Número de reseñas a lo largo de los años')

fig.tight_layout()

**Número de películas a lo largo de los años**

**Observaciones:**

Se observa un crecimiento progresivo en el número de películas a lo largo del tiempo, con un aumento especialmente marcado a partir de la segunda mitad del siglo XX. Este comportamiento sugiere una expansión de la industria cinematográfica y una mayor disponibilidad de títulos en los años más recientes. En los primeros años, la cantidad de películas es muy reducida, lo cual es consistente con el desarrollo temprano del cine.

**Número de reseñas a lo largo de los años**

**Observaciones:**

El número total de reseñas crece de forma significativa en los años más recientes, lo cual está alineado con el aumento en la cantidad de películas y con la popularización de plataformas digitales para dejar opiniones. La línea de promedio móvil de reseñas por película muestra una tendencia general al alza, lo que indica que, además de haber más películas, los usuarios participan más activamente dejando reseñas por título.

Veamos la distribución del número de reseñas por película con el conteo exacto y KDE (solo para saber cómo puede diferir del conteo exacto)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 5))

ax = axs[0]
dft = df_reviews.groupby('tconst')['review'].count() \
    .value_counts() \
    .sort_index()
dft.plot.bar(ax=ax)
ax.set_title('Gráfico de barras de #Reseñas por película')

ax = axs[1]
dft = df_reviews.groupby('tconst')['review'].count()
sns.kdeplot(dft, ax=ax)
ax.set_title('Gráfico KDE de #Reseñas por película')

fig.tight_layout()

**Distribución del número de reseñas por película (barras y KDE)**

**Observaciones:**

La distribución muestra que la mayoría de las películas cuentan con un número reducido de reseñas, mientras que solo un pequeño grupo de títulos concentra una gran cantidad de opiniones. Esto genera una distribución asimétrica con una cola larga hacia la derecha, lo cual es común en datos de popularidad. El gráfico KDE refuerza esta observación al mostrar una alta densidad en valores bajos y una disminución gradual conforme aumenta el número de reseñas.

In [ ]:
df_reviews['pos'].value_counts()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 4))

ax = axs[0]
dft = df_reviews.query('ds_part == "train"')['rating'].value_counts().sort_index()
dft = dft.reindex(index=np.arange(min(dft.index.min(), 1), max(dft.index.max(), 11))).fillna(0)
dft.plot.bar(ax=ax)
ax.set_ylim([0, 5000])
ax.set_title('El conjunto de entrenamiento: distribución de puntuaciones')

ax = axs[1]
dft = df_reviews.query('ds_part == "test"')['rating'].value_counts().sort_index()
dft = dft.reindex(index=np.arange(min(dft.index.min(), 1), max(dft.index.max(), 11))).fillna(0)
dft.plot.bar(ax=ax)
ax.set_ylim([0, 5000])
ax.set_title('El conjunto de prueba: distribución de puntuaciones')

fig.tight_layout()

**Distribución de puntuaciones en el conjunto de entrenamiento y prueba**

**Observaciones:**

Las distribuciones de puntuaciones en los conjuntos de entrenamiento y prueba son muy similares, lo que indica que la separación de los datos se realizó de manera adecuada. En ambos casos se observa una mayor concentración de puntuaciones extremas, especialmente en valores altos, lo que sugiere que los usuarios tienden a dejar reseñas muy positivas o muy negativas, en lugar de valoraciones neutrales.

Distribución de reseñas negativas y positivas a lo largo de los años para dos partes del conjunto de datos

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(16, 8), gridspec_kw=dict(width_ratios=(2, 1), height_ratios=(1, 1)))

ax = axs[0][0]

dft = df_reviews.query('ds_part == "train"').groupby(['start_year', 'pos'])['pos'].count().unstack()
dft.index = dft.index.astype('int')
dft = dft.reindex(index=np.arange(dft.index.min(), max(dft.index.max(), 2020))).fillna(0)
dft.plot(kind='bar', stacked=True, ax=ax)
ax.set_title('El conjunto de entrenamiento: número de reseñas de diferentes polaridades por año')

ax = axs[0][1]

dft = df_reviews.query('ds_part == "train"').groupby(['tconst', 'pos'])['pos'].count().unstack()
sns.kdeplot(dft[0], color='blue', label='negative', kernel='epa', ax=ax)
sns.kdeplot(dft[1], color='green', label='positive', kernel='epa', ax=ax)
ax.legend()
ax.set_title('El conjunto de entrenamiento: distribución de diferentes polaridades por película')

ax = axs[1][0]

dft = df_reviews.query('ds_part == "test"').groupby(['start_year', 'pos'])['pos'].count().unstack()
dft.index = dft.index.astype('int')
dft = dft.reindex(index=np.arange(dft.index.min(), max(dft.index.max(), 2020))).fillna(0)
dft.plot(kind='bar', stacked=True, ax=ax)
ax.set_title('El conjunto de prueba: número de reseñas de diferentes polaridades por año')

ax = axs[1][1]

dft = df_reviews.query('ds_part == "test"').groupby(['tconst', 'pos'])['pos'].count().unstack()
sns.kdeplot(dft[0], color='blue', label='negative', kernel='epa', ax=ax)
sns.kdeplot(dft[1], color='green', label='positive', kernel='epa', ax=ax)
ax.legend()
ax.set_title('El conjunto de prueba: distribución de diferentes polaridades por película')

fig.tight_layout()

**Número de reseñas positivas y negativas por año (train)**

**Observaciones:**

En el conjunto de entrenamiento se aprecia un aumento considerable en el número de reseñas tanto positivas como negativas con el paso del tiempo. Sin embargo, las reseñas positivas predominan de forma consistente, lo que indica un ligero desbalance de clases. Este comportamiento es relevante para el modelado, ya que puede influir en el desempeño de los modelos de clasificación.

**Distribución de polaridades por película (train)**

**Observaciones:**

La distribución muestra que, en promedio, las películas tienden a recibir más reseñas positivas que negativas. No obstante, ambas polaridades presentan una cola larga, lo que indica que existen películas con una gran cantidad de reseñas negativas o positivas. Esta variabilidad sugiere que el contenido de las reseñas es diverso y que el modelo deberá aprender a distinguir patrones lingüísticos más allá de la frecuencia.

**Número de reseñas positivas y negativas por año (test)**

**Observaciones:**

El conjunto de prueba presenta un patrón muy similar al del conjunto de entrenamiento, tanto en la evolución temporal como en la proporción entre reseñas positivas y negativas. Esto confirma que el conjunto de prueba es representativo del comportamiento general de los datos y permite evaluar el modelo de forma confiable.

**Distribución de polaridades por película (test)**

**Observaciones:**

Al igual que en el conjunto de entrenamiento, la distribución de polaridades por película en el conjunto de prueba muestra una mayor densidad de reseñas positivas. La similitud entre ambas distribuciones refuerza la consistencia de la partición de datos y reduce el riesgo de sesgos durante la evaluación del modelo.

## Procedimiento de evaluación

Composición de una rutina de evaluación que se pueda usar para todos los modelos en este proyecto

In [ ]:


import sklearn.metrics as metrics
def evaluate_model(model, train_features, train_target, test_features, test_target):

    eval_stats = {}

    fig, axs = plt.subplots(1, 3, figsize=(20, 6))

    for type, features, target in (('train', train_features, train_target), ('test', test_features, test_target)):

        eval_stats[type] = {}

        pred_target = model.predict(features)
        pred_proba = model.predict_proba(features)[:, 1]


        # F1
        f1_thresholds = np.arange(0, 1.01, 0.05)
        f1_scores = [metrics.f1_score(target, pred_proba>=threshold) for threshold in f1_thresholds]

        # ROC
        fpr, tpr, roc_thresholds = metrics.roc_curve(target, pred_proba)
        roc_auc = metrics.roc_auc_score(target, pred_proba)
        eval_stats[type]['ROC AUC'] = roc_auc

        # PRC
        precision, recall, pr_thresholds = metrics.precision_recall_curve(target, pred_proba)
        aps = metrics.average_precision_score(target, pred_proba)
        eval_stats[type]['APS'] = aps

        if type == 'train':
            color = 'blue'
        else:
            color = 'green'

        # Valor F1
        ax = axs[0]
        max_f1_score_idx = np.argmax(f1_scores)
        ax.plot(f1_thresholds, f1_scores, color=color, label=f'{type}, max={f1_scores[max_f1_score_idx]:.2f} @ {f1_thresholds[max_f1_score_idx]:.2f}')
        # establecer cruces para algunos umbrales
        for threshold in (0.2, 0.4, 0.5, 0.6, 0.8):
            closest_value_idx = np.argmin(np.abs(f1_thresholds-threshold))
            marker_color = 'orange' if threshold != 0.5 else 'red'
            ax.plot(f1_thresholds[closest_value_idx], f1_scores[closest_value_idx], color=marker_color, marker='X', markersize=7)
        ax.set_xlim([-0.02, 1.02])
        ax.set_ylim([-0.02, 1.02])
        ax.set_xlabel('threshold')
        ax.set_ylabel('F1')
        ax.legend(loc='lower center')
        ax.set_title(f'Valor F1')

        # ROC
        ax = axs[1]
        ax.plot(fpr, tpr, color=color, label=f'{type}, ROC AUC={roc_auc:.2f}')
        # establecer cruces para algunos umbrales
        for threshold in (0.2, 0.4, 0.5, 0.6, 0.8):
            closest_value_idx = np.argmin(np.abs(roc_thresholds-threshold))
            marker_color = 'orange' if threshold != 0.5 else 'red'
            ax.plot(fpr[closest_value_idx], tpr[closest_value_idx], color=marker_color, marker='X', markersize=7)
        ax.plot([0, 1], [0, 1], color='grey', linestyle='--')
        ax.set_xlim([-0.02, 1.02])
        ax.set_ylim([-0.02, 1.02])
        ax.set_xlabel('FPR')
        ax.set_ylabel('TPR')
        ax.legend(loc='lower center')
        ax.set_title(f'Curva ROC')

        # PRC
        ax = axs[2]
        ax.plot(recall, precision, color=color, label=f'{type}, AP={aps:.2f}')
        # establecer cruces para algunos umbrales
        for threshold in (0.2, 0.4, 0.5, 0.6, 0.8):
            closest_value_idx = np.argmin(np.abs(pr_thresholds-threshold))
            marker_color = 'orange' if threshold != 0.5 else 'red'
            ax.plot(recall[closest_value_idx], precision[closest_value_idx], color=marker_color, marker='X', markersize=7)
        ax.set_xlim([-0.02, 1.02])
        ax.set_ylim([-0.02, 1.02])
        ax.set_xlabel('recall')
        ax.set_ylabel('precision')
        ax.legend(loc='lower center')
        ax.set_title(f'PRC')

        eval_stats[type]['Accuracy'] = metrics.accuracy_score(target, pred_target)
        eval_stats[type]['F1'] = metrics.f1_score(target, pred_target)

    df_eval_stats = pd.DataFrame(eval_stats)
    df_eval_stats = df_eval_stats.round(2)
    df_eval_stats = df_eval_stats.reindex(index=('Accuracy', 'F1', 'APS', 'ROC AUC'))

    print(df_eval_stats)

    return



## Normalización

Suponemos que todos los modelos a continuación aceptan textos en minúsculas y sin dígitos, signos de puntuación, etc.

In [ ]:

# creamos una función para limpiar texto
import re

def clear_text(text):
    pattern = r"[^A-Za-z']"
    text = re.sub(pattern, ' ', text)
    text = " ".join(text.split())
    return text

df_reviews['review_norm'] = df_reviews['review'].apply(clear_text)

# No hacemos la conversión a minúsculas aquí debido a que es mejor hacerlo más adelante, al lematizar.


## División entrenamiento / prueba

Por fortuna, todo el conjunto de datos ya está dividido en partes de entrenamiento/prueba; 'ds_part' es el indicador correspondiente.

In [ ]:

df_reviews_train = df_reviews.query('ds_part == "train"').copy()
df_reviews_test = df_reviews.query('ds_part == "test"').copy()

target_train = df_reviews_train['pos']
target_test = df_reviews_test['pos']

print(df_reviews_train.shape)
print(df_reviews_test.shape)


## Trabajar con modelos

### Modelo 0 - Constante

In [ ]:
from sklearn.dummy import DummyClassifier

In [ ]:
features_train_dummy = np.zeros((len(target_train), 1))
features_test_dummy = np.zeros((len(target_test), 1))

dummy = DummyClassifier(strategy='most_frequent', random_state=42)
dummy.fit(features_train_dummy, target_train)

evaluate_model(dummy, features_train_dummy, target_train, features_test_dummy, target_test)

### Modelo 1 - NLTK, TF-IDF y LR

TF-IDF

In [ ]:
import nltk

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

from nltk.corpus import stopwords

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stop_words_nltk = set(stopwords.words('english'))


# tokenización y lematización avanzada con nltk
lemmatizer  = WordNetLemmatizer()

def tokenize_lemmatize_nltk(text):
    tokens = word_tokenize(text.lower())
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    return " ".join(lemmas)

# creamos la columna de características lematizadas con nltk en los conjuntos de entrenamiento y prueba
df_reviews_train['review_lemma_nltk'] = df_reviews_train['review_norm'].apply(tokenize_lemmatize_nltk)
df_reviews_test['review_lemma_nltk'] = df_reviews_test['review_norm'].apply(tokenize_lemmatize_nltk)

In [ ]:
# creamos el vectorizador para el modelo con nltk
tfidf_vectorizer_nltk = TfidfVectorizer(stop_words=stop_words_nltk, lowercase=False, ngram_range=(1, 2))
# creamos features_train y features_test para el modelo con nltk
train_corpus_nltk = df_reviews_train['review_lemma_nltk']
features_train_nltk = tfidf_vectorizer_nltk.fit_transform(train_corpus_nltk)

test_corpus_nltk = df_reviews_test['review_lemma_nltk']
features_test_nltk = tfidf_vectorizer_nltk.transform(test_corpus_nltk)

# creamos y entrenamos el modelo
lr_nltk = LogisticRegression(max_iter=1000, random_state=42)
lr_nltk.fit(features_train_nltk, target_train)

In [ ]:
# probamos el modelo
evaluate_model(lr_nltk, features_train_nltk, target_train, features_test_nltk, target_test)

### Modelo 2 - spaCy, TF-IDF y LR

In [ ]:
import spacy

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
stop_words_spacy = nlp.Defaults.stop_words

In [ ]:
# tokenización y lematización avanzada con spaCy
def tokenize_lemmatize_spacy(text):

    doc = nlp(text.lower())
    #tokens = [token.lemma_ for token in doc if not token.is_stop] (alternativa(eliminaríamos la variable stop_words_spacy))
    tokens = [token.lemma_ for token in doc]

    return " ".join(tokens)

# creamos la columna de características lematizadas con spaCy en los conjuntos de entrenamiento y prueba
df_reviews_train['review_lemma_spacy'] = df_reviews_train['review_norm'].apply(tokenize_lemmatize_spacy)
df_reviews_test['review_lemma_spacy'] = df_reviews_test['review_norm'].apply(tokenize_lemmatize_spacy)

In [ ]:
# creamos el vectorizador para el modelo con spaCy
tfidf_vectorizer_spacy = TfidfVectorizer(stop_words=stop_words_spacy, lowercase=False, ngram_range=(1, 2))
# creamos features_train y features_test para el modelo con spaCy
train_corpus_spacy = df_reviews_train['review_lemma_spacy']
features_train_spacy = tfidf_vectorizer_spacy.fit_transform(train_corpus_spacy)

test_corpus_spacy = df_reviews_test['review_lemma_spacy']
features_test_spacy = tfidf_vectorizer_spacy.transform(test_corpus_spacy)

# creamos y entrenamos el modelo
lr_spacy = LogisticRegression(max_iter=1000, random_state=42)
lr_spacy.fit(features_train_spacy, target_train)

In [ ]:
# probamos el modelo
evaluate_model(lr_spacy, features_train_spacy, target_train, features_test_spacy, target_test)

In [ ]:
print(features_train_nltk.shape, features_test_nltk.shape)
print(features_train_spacy.shape, features_test_spacy.shape)

### Modelo 3 - spaCy, TF-IDF y LGBMClassifier

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
# usaremos exactamente los mismos datos vectorizados del modelo anterior
# creamos y entrenamos el modelo
lgbm_spacy = LGBMClassifier(random_state=42, n_estimators=200, learning_rate=0.1)
lgbm_spacy.fit(features_train_spacy, target_train)

In [ ]:
# probamos el modelo
evaluate_model(lgbm_spacy, features_train_spacy, target_train, features_test_spacy, target_test)

###  Modelo 9 - BERT

In [ ]:
import torch
import transformers

In [ ]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
config = transformers.BertConfig.from_pretrained('bert-base-uncased')
model = transformers.BertModel.from_pretrained('bert-base-uncased')

In [ ]:
# VERSIÓN ORIGINAL

# def BERT_text_to_embeddings(texts, max_length=512, batch_size=100, force_device=None, disable_progress_bar=False):

#    enc = tokenizer(
#        texts.tolist() if hasattr(texts, "tolist") else texts,
#        add_special_tokens=True, padding='max_length',
#        truncation=True,
#        max_length=max_length,
#        return_attention_mask=True)
#    
#    ids_list = enc['input_ids']
#    attention_mask_list = enc['attention_mask']


#    if force_device is not None:
#        device = torch.device(force_device)
#    else:
#        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#    model.to(device)
#    if not disable_progress_bar:
#        print(f'Uso del dispositivo {device}.')

    # obtener insertados en lotes
    
#    embeddings = []
#    model.eval()
#    for i in tqdm(range(math.ceil(len(ids_list)/batch_size)), disable=disable_progress_bar):
#        ids_batch = torch.LongTensor(ids_list[batch_size*i:batch_size*(i+1)]).to(device)
#        attention_mask_batch = torch.LongTensor(attention_mask_list[batch_size*i:batch_size*(i+1)]).to(device)

#        with torch.no_grad():
#            batch_embeddings = model(input_ids=ids_batch, attention_mask=attention_mask_batch)
#        embeddings.append(batch_embeddings.last_hidden_state[:,0,:].detach().cpu().numpy())

#    return np.concatenate(embeddings)

In [ ]:
# # NOTA: La versión original de esta función (comentada arriba) tokenizaba todo el corpus antes de procesarlo.
# Sin embargo, en este entorno el kernel se reiniciaba por falta de memoria al manejar listas muy grandes.
# Por eso, usamos esta versión "streaming", que procesa los textos por lotes sin cargar todo a la RAM.
# La lógica es la misma, solo optimiza el uso de memoria.


def BERT_text_to_embeddings(texts, max_length=256, batch_size=8, force_device=None, disable_progress_bar=False):
    # Asegura lista de strings (si viene como Series)
    texts = texts.tolist() if hasattr(texts, "tolist") else texts

    # Selección de dispositivo
    device = torch.device(force_device) if force_device is not None else torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()
    if not disable_progress_bar:
        print(f'Uso del dispositivo {device}.')

    embeddings = []
    total = len(texts)

    # Procesa por lotes: tokeniza y pasa por el modelo SIN mantener todo en RAM
    for i in tqdm(range(0, total, batch_size), disable=disable_progress_bar):
        batch_texts = texts[i:i+batch_size]

        enc = tokenizer(
            batch_texts,
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            max_length=max_length,
            return_attention_mask=True,
            return_tensors='pt'     # <- clave: devuelve tensores PyTorch directamente
        )

        input_ids = enc['input_ids'].to(device)
        attention_mask = enc['attention_mask'].to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            cls = outputs.last_hidden_state[:, 0, :].detach().cpu().numpy().astype(np.float32)

        embeddings.append(cls)

    return np.concatenate(embeddings, axis=0)

In [ ]:
# Intentamos cargar embeddings ya calculados para evitar recalcular BERT
try:
    with np.load('features_bert.npz') as data:
        features_train_bert = data['features_train_bert']
        features_test_bert = data['features_test_bert']
    print("Embeddings BERT cargados desde archivo features_bert.npz.")
except FileNotFoundError:
    print("features_bert.npz no existe. Generando embeddings con BERT (200 train / 200 test)...")
    features_train_bert = BERT_text_to_embeddings(df_reviews_train['review_norm'][:200])
    features_test_bert = BERT_text_to_embeddings(df_reviews_test['review_norm'][:200])

    np.savez_compressed(
        'features_bert.npz',
        features_train_bert=features_train_bert,
        features_test_bert=features_test_bert
    )
    print("Embeddings guardados en features_bert.npz.")

In [ ]:
# ¡Atención! La ejecución de BERT para miles de textos puede llevar mucho tiempo en la CPU, al menos varias horas

In [ ]:
print(df_reviews_train['review_norm'].shape)
print(features_train_bert.shape)
print(target_train.shape)
print()
print(df_reviews_test['review_norm'].shape)
print(features_test_bert.shape)
print(target_test.shape)

In [ ]:
# creamos y entrenamos el modelo
lr_bert = LogisticRegression(max_iter=1000, random_state=42)
lr_bert.fit(features_train_bert, target_train.iloc[:features_train_bert.shape[0]])

In [ ]:
# probamos el modelo
evaluate_model(lr_bert, features_train_bert, target_train.iloc[:features_train_bert.shape[0]], features_test_bert, target_test.iloc[:features_test_bert.shape[0]])

## Mis reseñas

In [ ]:
# puedes eliminar por completo estas reseñas y probar tus modelos en tus propias reseñas; las que se muestran a continuación son solo ejemplos

my_reviews = pd.DataFrame([
    'I did not simply like it, not my kind of movie.',
    'Well, I was bored and felt asleep in the middle of the movie.',
    'I was really fascinated with the movie',
    'Even the actors looked really old and disinterested, and they got paid to be in the movie. What a soulless cash grab.',
    'I didn\'t expect the reboot to be so good! Writers really cared about the source material',
    'The movie had its upsides and downsides, but I feel like overall it\'s a decent flick. I could see myself going to see it again.',
    'What a rotten attempt at a comedy. Not a single joke lands, everyone acts annoying and loud, even kids won\'t like this!',
    'Launching on Netflix was a brave move & I really appreciate being able to binge on episode after episode, of this exciting intelligent new drama.'
], columns=['review'])

"""
my_reviews = pd.DataFrame([
    'Simplemente no me gustó, no es mi tipo de película.',
    'Bueno, estaba aburrido y me quedé dormido a media película.',
    'Estaba realmente fascinada con la película',
    'Hasta los actores parecían muy viejos y desinteresados, y les pagaron por estar en la película. Qué robo tan desalmado.',
    '¡No esperaba que el relanzamiento fuera tan bueno! Los escritores realmente se preocuparon por el material original',
    'La película tuvo sus altibajos, pero siento que, en general, es una película decente. Sí la volvería a ver',
    'Qué pésimo intento de comedia. Ni una sola broma tiene sentido, todos actúan de forma irritante y ruidosa, ¡ni siquiera a los niños les gustará esto!',
    'Fue muy valiente el lanzamiento en Netflix y realmente aprecio poder seguir viendo episodio tras episodio de este nuevo drama tan emocionante e inteligente.'
], columns=['review'])
"""

my_reviews['review_norm'] = my_reviews['review'].apply(clear_text)

### Modelo 1 (LR+NLTK)

In [ ]:
# definimos el corpus normalizado
corpus_norm = my_reviews['review_norm']
# hacemos tokenización y lematización avanzada con nltk
my_reviews['review_lemma_nltk'] = corpus_norm.apply(tokenize_lemmatize_nltk)
corpus_lemma_nltk = my_reviews['review_lemma_nltk']
# creamos las features con nltk
my_reviews_features_nltk = tfidf_vectorizer_nltk.transform(corpus_lemma_nltk)
# obtenemos las probabilidades de que cada reseña sea positiva
my_reviews_pred_proba_lr_nltk = lr_nltk.predict_proba(my_reviews_features_nltk)[:, 1]
# mostramos cada reseña con su probabilidad de ser positiva
for i, review in enumerate(corpus_norm.str.slice(0, 100)):
    print(f'{my_reviews_pred_proba_lr_nltk[i]:.2f}:  {review}')

### Modelo 2 (LR+spaCy)

In [ ]:
# definimos el corpus normalizado
corpus_norm = my_reviews['review_norm']
# hacemos tokenización y lematización avanzada con spacy
my_reviews['review_lemma_spacy'] = corpus_norm.apply(tokenize_lemmatize_spacy)
corpus_lemma_spacy = my_reviews['review_lemma_spacy']
# creamos las features con spacy
my_reviews_features_spacy = tfidf_vectorizer_spacy.transform(corpus_lemma_spacy)
# obtenemos las probabilidades de que cada reseña sea positiva
my_reviews_pred_proba_lr_spacy = lr_spacy.predict_proba(my_reviews_features_spacy)[:, 1]
# mostramos cada reseña con su probabilidad de ser positiva
for i, review in enumerate(corpus_norm.str.slice(0, 100)):
    print(f'{my_reviews_pred_proba_lr_spacy[i]:.2f}:  {review}')

### Modelo 3 (LGBMClassifier+spaCy)

In [ ]:
# Nota: en este paso no repetimos la lematización ni la vectorización, ya que este modelo usa las mismas features TF-IDF creadas para el modelo anterior.

# obtenemos las probabilidades de que cada reseña sea positiva
my_reviews_pred_proba_lgbm_spacy = lgbm_spacy.predict_proba(my_reviews_features_spacy)[:, 1]
# mostramos cada reseña con su probabilidad de ser positiva
for i, review in enumerate(corpus_norm.str.slice(0, 100)):
    print(f'{my_reviews_pred_proba_lgbm_spacy[i]:.2f}:  {review}')

### Modelo BERT

In [ ]:
# creamos las features con BERT
my_reviews_features_bert = BERT_text_to_embeddings(corpus_norm, disable_progress_bar=False)
# obtenemos las probabilidades de que cada reseña sea positiva
my_reviews_pred_proba_bert = lr_bert.predict_proba(my_reviews_features_bert)[:, 1]
# mostramos cada reseña con su probabilidad de ser positiva
for i, review in enumerate(corpus_norm.str.slice(0, 100)):
    print(f'{my_reviews_pred_proba_bert[i]:.2f}:  {review}')

## Conclusiones

**Modelo Logistic Regression + NLTK**

Este modelo muestra un rendimiento excelente, con valores de F1 y ROC AUC muy altos tanto en entrenamiento como en prueba. La pequeña diferencia entre ambos conjuntos indica que el modelo generaliza bien y no presenta sobreajuste significativo. Esto demuestra que el modelo ha aprendido correctamente las relaciones entre las palabras y el sentimiento de las reseñas, interpretando de forma precisa tanto los textos positivos como los negativos.

**Modelo Logistic Regression + spaCy**

Este modelo obtuvo resultados muy similares al de NLTK, apenas ligeramente menores en el conjunto de prueba. Esto sugiere que ambas estrategias de procesamiento del lenguaje (NLTK y spaCy) son igualmente efectivas para este corpus, y que el rendimiento depende más de la representación TF-IDF y el modelo lineal que de la librería usada para la lematización.

**Modelo LGBMClassifier + spaCy**

Este modelo también logra un rendimiento alto, aunque ligeramente inferior al de la regresión logística. Esto puede deberse a que los modelos de tipo árbol como LightGBM requieren más datos para superar a los modelos lineales en tareas de texto, donde los vectores TF-IDF suelen ser muy dispersos. Aun así, su rendimiento sigue siendo sólido y demuestra que puede capturar bien las relaciones no lineales en las reseñas.

**Modelo BERT**

Este modelo sí mostró claramente un sobreajuste, al alcanzar un F1 de 100 % en entrenamiento, y un 75% en el conjunto de prueba. Esto puede deberse a que los embeddings se generaron a partir de solo 200 reseñas, lo que limita la capacidad de generalización del modelo. A pesar de ello, el modelo aún logra predicciones razonables en el conjunto de prueba, mostrando el potencial de BERT incluso con datos limitados.

**Mis reseñas**

Al aplicar los modelos entrenados a mis reseñas, podemos observar que cada uno asigna una probabilidad de positividad distinta a los textos.
En general, con los modelos NLTK y spaCy, las reseñas con un tono claramente positivo tienden a recibir valores más altos (por encima de 0.6), mientras que las reseñas con un tono negativo suelen tener valores más bajos (por debajo de 0.4). LGBM varía un poco más, pero sigue la misma tendencia.

Sin embargo, hay algunos casos donde el modelo asigna probabilidades intermedias o contradictorias (por ejemplo, frases positivas con valores menores a 0.5), lo que indica que no siempre interpreta correctamente el tono emocional o el contexto del texto. Esto es normal, ya que las probabilidades reflejan la confianza del modelo, no necesariamente la verdad objetiva. BERT se comporta más errático con pocas muestras, lo cual confirma su sobreajuste por datos limitados.

En conjunto, las probabilidades permiten observar qué tan seguro está el modelo de que cada reseña sea positiva o negativa, y sirven para entender el comportamiento del modelo en textos nuevos.